In [3]:
import pandas as pd

# データを読み込む
data = pd.read_excel("chatgpt_train_data.xlsx")

# 最初の数行を表示して、データの内容を確認する
data.head()

,Mean CTDIvol,DLP,study_date,id,age,gender,height_cm,weight_kg,scan_area,scan_method,...,category_bmi,age_era,bmi_body_surface_area_category,age_weight_kg_category,scan_area_method_combine,scan_area_method_bmi_catgory_combine,bmi_bsa_cat_scan_area_method_combine,age_weight_cat_scan_area_method_combine,predicted_exposure_time,predicted_pitch_factor
0,5.86,223.82,2021-01-01,12585,86,F,150.0,49.1,胸部CT,単純,...,normal weight,80,8,8,胸部CT_単純,胸部CT_単純_normal weight,8胸部CT_単純,8胸部CT_単純,0.5,1.38
1,7.41,327.55,2021-01-01,2145609,61,M,163.0,74.0,胸部CT,単純,...,obesity class1,60,2,7,胸部CT_単純,胸部CT_単純_obesity class1,2胸部CT_単純,7胸部CT_単純,0.5,1.00
2,5.13,239.47,2021-01-02,12615,46,M,178.0,68.0,胸部CT,単純,...,normal weight,40,6,5,胸部CT_単純,胸部CT_単純_normal weight,6胸部CT_単純,5胸部CT_単純,0.5,1.38
3,7.25,342.20,2021-01-02,12616,52,M,182.0,70.0,胸部CT,単純,...,normal weight,50,6,5,胸部CT_単純,胸部CT_単純_normal weight,6胸部CT_単純,5胸部CT_単純,0.5,1.38
4,11.56,534.33,2021-01-03,9595120,52,M,175.0,112.0,胸部CT,単純,...,obesity class3,50,4,2,胸部CT_単純,胸部CT_単純_obesity class3,4胸部CT_単純,2胸部CT_単純,0.5,0.51


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error

# ターゲット変数と説明変数を分割
X = data.drop(columns=["Mean CTDIvol"])
y = data["Mean CTDIvol"]

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# DummyRegressorを初期化し、学習
dummy_regressor = DummyRegressor(strategy="mean")
dummy_regressor.fit(X_train, y_train)

# テストデータでの予測を行う
y_pred = dummy_regressor.predict(X_test)

# Mean Squared Error (MSE)を計算
mse = mean_squared_error(y_test, y_pred)
mse


71.93742495437638

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 説明変数とターゲット変数を選択
features = ['height_cm', 'weight_kg', 'age', 'gender', 'scan_area', 'scan_method']
X = data[features]
y = data['Mean CTDIvol']

# カテゴリカル変数をダミー変数に変換するためのトランスフォーマを作成
categorical_features = ['gender', 'scan_area', 'scan_method']
column_transformer = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), categorical_features)
    ], 
    remainder='passthrough'
)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 線形回帰モデルを作成し、訓練
pipeline = Pipeline([
    ('transformer', column_transformer),
    ('regressor', LinearRegression())
])
pipeline.fit(X_train, y_train)

# テストデータで予測
y_pred = pipeline.predict(X_test)

# MAEを計算
mae = mean_absolute_error(y_test, y_pred)
mae


2.0179243599088084

In [8]:
from catboost import CatBoostRegressor

# カテゴリカル変数の位置を指定
cat_features_indices = [X.columns.get_loc(col) for col in categorical_features]

# CatBoostRegressorのインスタンスを作成
catboost_regressor = CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.05, loss_function='RMSE', verbose=0)

# モデルを学習
catboost_regressor.fit(X_train, y_train, cat_features=cat_features_indices)

# テストデータで予測
y_pred_catboost = catboost_regressor.predict(X_test)

# 予測の誤差を計算
mae = mean_absolute_error(y_test, y_pred_catboost) 
mae

1.874975381167748

In [9]:
import xgboost as xgb

# カテゴリカル変数をダミー変数に変換
X_encoded = pd.get_dummies(X, columns=categorical_features, drop_first=True)

# 訓練データとテストデータに再分割
X_train_encoded, X_test_encoded = train_test_split(X_encoded, test_size=0.2, random_state=42)

# XGBoostの回帰モデルを学習
xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=1000)
xgb_regressor.fit(X_train_encoded, y_train)

# テストデータで予測
y_pred_xgb = xgb_regressor.predict(X_test_encoded)

# 予測の誤差を計算
errors_xgb = y_test - y_pred_xgb

# mae
mae = mean_absolute_error(y_test, y_pred_xgb)
mae

1.8652969862467117

In [12]:
import lightgbm as lgb

# カテゴリカル変数をcategory型に変換
for feature in categorical_features:
    X_train[feature] = X_train[feature].astype('category')
    X_test[feature] = X_test[feature].astype('category')

# LightGBMのデータセットを作成
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features, free_raw_data=False)

# ハイパーパラメータの設定
params = {
    'objective': 'regression',
    'metric': 'rmse',
}

# モデルを学習
lgb_regressor = lgb.train(params, train_data, num_boost_round=1000)

# テストデータで予測
y_pred_lgb = lgb_regressor.predict(X_test)

# 予測の誤差を計算
errors_lgb = y_test - y_pred_lgb

# 誤差の最大値を取得
max_error_lgb = errors_lgb.abs().max()
max_error_lgb

mae = mean_absolute_error(y_test, y_pred_lgb)
mae

/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000384 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 586
[LightGBM] [Info] Number of data points in the train set: 6843, number of used features: 6
[LightGBM] [Info] Start training from score 13.669070


1.9422019294348405

In [14]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import LeaveOneOut
import time

# データの読み込み
data = pd.read_excel("chatgpt_train_data.xlsx")

# 特徴量とターゲット変数の選択
features = ['height_cm', 'weight_kg', 'age', 'gender', 'scan_area', 'scan_method']
X = data[features]
y = data['Mean CTDIvol']

# カテゴリカル変数の定義
categorical_features = ['gender', 'scan_area', 'scan_method']
for feature in categorical_features:
    X[feature] = X[feature].astype('category')

# LOOCVを初期化
loo = LeaveOneOut()
errors_loocv_mae = []
start_time_mae = time.time()

# LOOCVでの学習と評価を実施
for train_index, test_index in loo.split(X):
    X_train_loo, X_test_loo = X.iloc[train_index], X.iloc[test_index]
    y_train_loo, y_test_loo = y.iloc[train_index], y.iloc[test_index]
    
    train_data_loo = lgb.Dataset(X_train_loo, label=y_train_loo, categorical_feature=categorical_features, free_raw_data=False)
    
    lgb_regressor_loo = lgb.train(params, train_data_loo, num_boost_round=10)  # Reduce the number of rounds for quicker evaluation
    y_pred_loo = lgb_regressor_loo.predict(X_test_loo)
    
    error_loo_mae = abs(y_test_loo.values[0] - y_pred_loo[0])
    errors_loocv_mae.append(error_loo_mae)

end_time_mae = time.time()
elapsed_time_mae = end_time_mae - start_time_mae

# 平均絶対誤差 (MAE) を計算
mae_loocv = sum(errors_loocv_mae) / len(errors_loocv_mae)

mae_loocv, elapsed_time_mae


/tmp/ipykernel_98/2107375852.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = X[feature].astype('category')
/tmp/ipykernel_98/2107375852.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[feature] = X[feature].astype('category')
/tmp/ipykernel_98/2107375852.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000079 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 8553, number of used features: 6
[LightGBM] [Info] Start training from score 13.635209
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000080 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 8553, number of used features: 6
[LightGBM] [Info] Start training from score 13.635028
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000259 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not 

/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warni

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 8553, number of used features: 6
[LightGBM] [Info] Start training from score 13.633987
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000133 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 8553, number of used features: 6
[LightGBM] [Info] Start training from score 13.635234
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000536 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Tot

/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000221 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 8553, number of used features: 6
[LightGBM] [Info] Start training from score 13.634135
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000120 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 8553, number of used features: 6
[LightGBM] [Info] Start training from score 13.634046
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001095 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Tot

/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 8553, number of used features: 6
[LightGBM] [Info] Start training from score 13.634110
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000296 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 585
[LightGBM] [Info] Number of data points in the train set: 8553, number of used features: 6
[LightGBM] [Info] Start training from score 13.633161
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000109 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not 

/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


KeyboardInterrupt: 

In [ ]:
import xgboost as xgb
from sklearn.model_selection import LeaveOneOut
import time
import pandas as pd

# データの読み込み
data = pd.read_excel("your_data_path_here.xlsx")

# 特徴量とターゲット変数の選択
features = ['height_cm', 'weight_kg', 'age', 'gender', 'scan_area', 'scan_method']
X = pd.get_dummies(data[features], drop_first=True)  # カテゴリカル変数をダミー変数に変換
y = data['Mean CTDIvol']

loo = LeaveOneOut()
errors_xgb_mae = []
start_time_xgb = time.time()

for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_xgb = xgb.XGBRegressor(objective='reg:squarederror')
    model_xgb.fit(X_train, y_train)

    y_pred = model_xgb.predict(X_test)
    error = abs(y_test.iloc[0] - y_pred[0])
    errors_xgb_mae.append(error)

mae_xgb = sum(errors_xgb_mae) / len(errors_xgb_mae)
elapsed_time_xgb = time.time() - start_time_xgb
print(f"XGBoost MAE: {mae_xgb}, Time taken: {elapsed_time_xgb} seconds")


In [ ]:
import catboost as cb
from sklearn.model_selection import LeaveOneOut
import time
import pandas as pd

# データの読み込み
data = pd.read_excel("your_data_path_here.xlsx")

# 特徴量とターゲット変数の選択
features = ['height_cm', 'weight_kg', 'age', 'gender', 'scan_area', 'scan_method']
X = data[features]
y = data['Mean CTDIvol']

cat_features = ['gender', 'scan_area', 'scan_method']

loo = LeaveOneOut()
errors_cb_mae = []
start_time_cb = time.time()

for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model_cb = cb.CatBoostRegressor(verbose=0)
    model_cb.fit(X_train, y_train, cat_features=cat_features)

    y_pred = model_cb.predict(X_test)
    error = abs(y_test.iloc[0] - y_pred[0])
    errors_cb_mae.append(error)

mae_cb = sum(errors_cb_mae) / len(errors_cb_mae)
elapsed_time_cb = time.time() - start_time_cb
print(f"CatBoost MAE: {mae_cb}, Time taken: {elapsed_time_cb} seconds")
